In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
code = "select * from customer where id = 10"
# code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = AutoModel.from_pretrained("microsoft/graphcodebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize(code)
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]

tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(context_embeddings.shape)
print("\n")
print(len(tokens_ids))
print("\n")
print(context_embeddings)

torch.Size([1, 14, 768])


14


tensor([[[ 4.2571e-01,  1.5623e-01, -4.3657e-02,  ...,  2.6865e-01,
          -4.8247e-01,  1.1124e-01],
         [ 8.5409e-01, -6.5083e-01,  4.7403e-01,  ..., -2.3029e-01,
           3.5549e-01,  5.5711e-02],
         [ 2.4321e-01,  2.2712e-01, -1.0862e-01,  ..., -1.6620e-01,
          -2.8561e-01,  1.6535e-01],
         ...,
         [ 4.2238e-01, -4.7635e-01, -6.0525e-04,  ..., -5.3228e-02,
          -4.5180e-02,  1.1111e+00],
         [ 1.4651e-01, -2.6012e-01,  2.4449e-01,  ...,  8.0713e-01,
          -2.9616e-02,  9.4971e-01],
         [ 4.2580e-01,  1.5634e-01, -4.3713e-02,  ...,  2.6890e-01,
          -4.8293e-01,  1.1113e-01]]], grad_fn=<NativeLayerNormBackward0>)


In [ ]:
final_embedding = torch.sum(context_embeddings, dim=1)
print(final_embedding.shape)

torch.Size([1, 768])


In [ ]:
final_embedding[0][:10]

tensor([ 5.7854, -2.5959, -1.2843,  5.0936, -1.1977, -4.1835, -1.4712,  3.0721,
         4.1025, -3.2835], grad_fn=<SliceBackward0>)

### Resize Embedding to 512 and 1024

In [ ]:
# Approch 1

In [36]:
# prompt: increase embedding from 768 to 1024 no padding

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn

code = "select * from customer where id = 10"
target_embedding_size = 512  # or 1024

tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = AutoModel.from_pretrained("microsoft/graphcodebert-base")

# Define a linear layer to increase embedding dimension
embedding_increase_layer = nn.Linear(768, target_embedding_size)

nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize(code)
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)

context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

# Apply the linear layer to increase the embedding dimension
context_embeddings = embedding_increase_layer(context_embeddings)

final_embedding = torch.mean(context_embeddings, dim=1)

print(final_embedding.shape)
print(len(tokens_ids))
final_embedding[0][:10]

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 512])
14


tensor([-0.2065, -0.1936, -0.3923,  0.5156,  0.1979, -0.1955,  0.1809, -0.1376,
         0.1576, -0.1373], grad_fn=<SliceBackward0>)

In [ ]:
# Approach 2

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

code = "select * from customer where id = 10"
target_embedding_size = 512  # or 1024

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = AutoModel.from_pretrained("microsoft/graphcodebert-base")

# Tokenize the sentence
inputs = tokenizer(code, return_tensors='pt')

# Get the model output
with torch.no_grad():
    outputs = model(**inputs)

# Max pooling to get sentence embedding
sentence_embedding, _ = torch.max(outputs.last_hidden_state, dim=1)

# Create a linear layer to reduce the embedding dimension
dim_reduction_layer = nn.Linear(768, target_embedding_size)

# Apply the linear layer to the sentence embedding
final_embedding = dim_reduction_layer(sentence_embedding)

print(final_embedding.shape)  # Output: torch.Size([1, 512])


Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 512])


In [37]:
# https://github.com/microsoft/CodeBERT/blob/master/GraphCodeBERT/refinement/README.md